In [1]:
import pandas as pd
from lxml import html
import numpy as np
from matplotlib import pyplot as plt
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.ensemble import RandomForestClassifier
import gensim
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter,defaultdict
from string import punctuation
import os
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
%matplotlib inline

morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return ' '.join(words)

def tokenize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]

    return ' '.join(words)


In [78]:
corpus_xml = html.fromstring(open('./data/paraphrases.xml', 'rb').read())
texts_1 = []
texts_2 = []
classes = []

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
data = pd.DataFrame({'text_1':texts_1, 'text_2':texts_2, 'label':classes})

In [79]:
data.head()

,label,text_1,text_2
0,0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...
1,0,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...
2,0,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...
3,-1,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...
4,0,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...


In [7]:
data['text_1_norm'] = data['text_1'].apply(normalize)
data['text_2_norm'] = data['text_2'].apply(normalize)

In [8]:
data.head()

,label,text_1,text_2,text_1_norm,text_2_norm
0,0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,полицейский разрешить стрелять поражение гражд...,полиция мочь разрешить стрелять хулиган травма...
1,0,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,право полицейский проникновение жилища решить ...,правило внесудебный проникновение полицейский ...
2,0,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,президент египет ввести чрезвычайный положение...,власть египет угрожать ввести страна чрезвычай...
3,-1,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,вернуться сирия россиянин волновать вопрос тру...,самолёт мчс вывезти россиянин разрушить сирия
4,0,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,москва сирия вернуться 2 самолёт мчс россиянин...,самолёт мчс вывезти россиянин разрушить сирия


In [24]:
DF = pd.read_csv('./data/news_texts.csv').fillna('')

In [25]:
DF.head(3)

,content,content_norm
0,Канцлер Германии Ангела Меркель в ходе брифинг...,канцлер германия ангел меркель ход брифинг пре...
1,Российские и белорусские войска успешно заверш...,российский белорусский войско успешно завершит...
2,"Дзюба, Шатов и Анюков оказались не нужны «Зени...",дзюба шат анюк оказаться нужный зенит российск...


In [26]:
FT_N = gensim.models.FastText([text.split() for text in DF['content_norm'].values], 
                              size=300, min_n=4, max_n=8, workers=8, seed=23)
FT_O = gensim.models.FastText([text.split() for text in DF['content'].values], 
                              size=300, min_n=4, max_n=8, workers=8, seed=23)
W2V = gensim.models.Word2Vec([text.split() for text in DF['content_norm'].values], size=300, sg=1, 
                             workers=8, seed=23)

In [27]:
def get_embedding(text, model, dim):
    text = text.split()
    
    words = Counter(text)
    total = len(text)
    vectors = np.zeros((len(words), dim))
    
    for i,word in enumerate(words):
        try:
            v = model[word]
            vectors[i] = v*(words[word]/total) # просто умножаем вектор на частоту
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector

In [28]:
# W2V
dim = 300
X_text_1_w2v = np.zeros((len(data['text_1_norm']), dim))
X_text_2_w2v = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v[i] = get_embedding(text, W2V, dim)
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v[i] = get_embedding(text, W2V, dim)

/home/glznv/.local/lib/python3.5/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [29]:
from sklearn.metrics.pairwise import cosine_similarity

In [32]:
cosine_similarity([X_text_1_w2v[0]], [X_text_2_w2v[0]])[0,0]

0.7978847389486156

In [34]:
data['W2V_none'] = [cosine_similarity([X_text_1_w2v[i]], [X_text_2_w2v[i]])[0,0] for i in range(len(X_text_1_w2v))]

In [39]:
from gensim.models.tfidfmodel import TfidfModel
from gensim.matutils import sparse2full
from gensim.corpora import Dictionary

def vectors(docs, model):
    docs = [doc.split() for doc in docs]
    docs_dict = Dictionary(docs)
    docs_dict.filter_extremes(no_below=20, no_above=0.8)
    docs_dict.compactify()

    docs_corpus = [docs_dict.doc2bow(doc) for doc in docs]
    model_tfidf = TfidfModel(docs_corpus, id2word=docs_dict)
    docs_tfidf  = model_tfidf[docs_corpus]
    docs_vecs   = np.vstack([sparse2full(c, len(docs_dict)) for c in docs_tfidf])
    tfidf_emb_vecs = np.vstack([model[docs_dict[i]] if docs_dict[i] in model else np.zeros(300) 
                                for i in range(len(docs_dict)) ])
    docs_emb = np.dot(docs_vecs, tfidf_emb_vecs) 
    return docs_emb

In [41]:
w2v_weght_1 = vectors(data['text_1_norm'], W2V)
w2v_weght_2 = vectors(data['text_2_norm'], W2V)
data['W2V_TF'] = [cosine_similarity([w2v_weght_1[i]], [w2v_weght_2[i]])[0,0] for i in range(len(w2v_weght_1))]

/home/glznv/.local/lib/python3.5/site-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  app.launch_new_instance()
/home/glznv/.local/lib/python3.5/site-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


In [43]:
# FT_N
dim = 300
X_text_1_FT_N = np.zeros((len(data['text_1_norm']), dim))
X_text_2_FT_N = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_FT_N[i] = get_embedding(text, FT_N, dim)
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_FT_N[i] = get_embedding(text, FT_N, dim)
    
data['FT_N_none'] = [cosine_similarity([X_text_1_w2v[i]], [X_text_2_w2v[i]])[0,0] for i in range(len(X_text_1_w2v))] 

/home/glznv/.local/lib/python3.5/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [44]:
weight_1 = vectors(data['text_1_norm'], FT_N)
weight_2 = vectors(data['text_2_norm'], FT_N)
data['FT_N_TF'] = [cosine_similarity([weight_1[i]], [weight_2[i]])[0,0] for i in range(len(weight_2))]

/home/glznv/.local/lib/python3.5/site-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  app.launch_new_instance()
/home/glznv/.local/lib/python3.5/site-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


In [45]:
# FT_O
dim = 300
X_text_1_FT_O = np.zeros((len(data['text_1']), dim))
X_text_2_FT_O = np.zeros((len(data['text_2']), dim))

for i, text in enumerate(data['text_1'].values):
    X_text_1_FT_O[i] = get_embedding(text.lower(), FT_O, dim)
for i, text in enumerate(data['text_2'].values):
    X_text_2_FT_O[i] = get_embedding(text.lower(), FT_O, dim)
    
data['FT_O_none'] = [cosine_similarity([X_text_1_FT_O[i]], [X_text_2_FT_O[i]])[0,0] for i in range(len(X_text_1_FT_O))] 

weight_1 = vectors(data['text_1'], FT_O)
weight_2 = vectors(data['text_2'], FT_O)
data['FT_O_TF'] = [cosine_similarity([weight_1[i]], [weight_2[i]])[0,0] for i in range(len(weight_2))]

/home/glznv/.local/lib/python3.5/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.
/home/glznv/.local/lib/python3.5/site-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  app.launch_new_instance()
/home/glznv/.local/lib/python3.5/site-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


In [46]:
data.head()

,label,text_1,text_2,text_1_norm,text_2_norm,W2V_none,W2V_TF,FT_N_none,FT_N_TF,FT_O_none,FT_O_TF
0,0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,полицейский разрешить стрелять поражение гражд...,полиция мочь разрешить стрелять хулиган травма...,0.797885,0.772642,0.797885,0.776446,0.892355,0.957886
1,0,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,право полицейский проникновение жилища решить ...,правило внесудебный проникновение полицейский ...,0.803657,0.566142,0.803657,0.446428,0.863612,0.702982
2,0,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,президент египет ввести чрезвычайный положение...,власть египет угрожать ввести страна чрезвычай...,0.898032,0.808042,0.898032,0.667296,0.943043,0.842504
3,-1,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,вернуться сирия россиянин волновать вопрос тру...,самолёт мчс вывезти россиянин разрушить сирия,0.675170,0.677628,0.675170,0.542383,0.808040,0.933931
4,0,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,москва сирия вернуться 2 самолёт мчс россиянин...,самолёт мчс вывезти россиянин разрушить сирия,0.867209,0.873730,0.867209,0.690162,0.495889,0.512894


In [47]:
cv = CountVectorizer(min_df=3, max_df=0.4, max_features=10000)
cv.fit(DF['content_norm'])
tf = TfidfVectorizer(ngram_range=(1,3), min_df=5, max_df=0.8, max_features=10000)
tf.fit(DF['content_norm'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.8, max_features=10000, min_df=5,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [50]:
CV_1 = cv.transform(data['text_1_norm'])
CV_2 = cv.transform(data['text_2_norm'])
TF_1 = tf.transform(data['text_1_norm'])
TF_2 = tf.transform(data['text_2_norm'])

In [55]:
nmf_cv = NMF(300)
nmf_cv.fit(cv.transform(DF['content_norm']))

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=300, random_state=None, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [56]:
svd_cv = TruncatedSVD(300)
svd_cv.fit(cv.transform(DF['content_norm']))

TruncatedSVD(algorithm='randomized', n_components=300, n_iter=5,
       random_state=None, tol=0.0)

In [57]:
nmf_tf = NMF(300)
nmf_tf.fit(tf.transform(DF['content_norm']))

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=300, random_state=None, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [58]:
svd_tf = TruncatedSVD(300)
svd_tf.fit(tf.transform(DF['content_norm']))

TruncatedSVD(algorithm='randomized', n_components=300, n_iter=5,
       random_state=None, tol=0.0)

In [60]:
nmf_cv_1 = nmf_cv.transform(cv.transform(data['text_1_norm']))
nmf_cv_2 = nmf_cv.transform(cv.transform(data['text_2_norm']))
nmf_tf_1 = nmf_tf.transform(tf.transform(data['text_1_norm']))
nmf_tf_2 = nmf_tf.transform(tf.transform(data['text_2_norm']))
svd_cv_1 = svd_cv.transform(cv.transform(data['text_1_norm']))
svd_cv_2 = svd_cv.transform(cv.transform(data['text_2_norm']))
svd_tf_1 = svd_tf.transform(tf.transform(data['text_1_norm']))
svd_tf_2 = svd_tf.transform(tf.transform(data['text_2_norm']))

In [61]:
data['NMF_cv'] = [cosine_similarity([nmf_cv_1[i]], [nmf_cv_2[i]])[0,0] for i in range(len(nmf_cv_2))]
data['NMF_tf'] = [cosine_similarity([nmf_tf_1[i]], [nmf_tf_2[i]])[0,0] for i in range(len(nmf_tf_2))]
data['SVD_cv'] = [cosine_similarity([svd_cv_1[i]], [svd_cv_2[i]])[0,0] for i in range(len(svd_cv_2))]
data['SVD_tf'] = [cosine_similarity([svd_tf_1[i]], [svd_tf_2[i]])[0,0] for i in range(len(svd_tf_2))]

In [62]:
data.head()

,label,text_1,text_2,text_1_norm,text_2_norm,W2V_none,W2V_TF,FT_N_none,FT_N_TF,FT_O_none,FT_O_TF,NMF_cv,NMF_tf,SVD_cv,SVD_tf
0,0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,полицейский разрешить стрелять поражение гражд...,полиция мочь разрешить стрелять хулиган травма...,0.797885,0.772642,0.797885,0.776446,0.892355,0.957886,0.355364,0.980594,0.275518,0.636863
1,0,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,право полицейский проникновение жилища решить ...,правило внесудебный проникновение полицейский ...,0.803657,0.566142,0.803657,0.446428,0.863612,0.702982,0.224124,0.988423,0.084903,0.691494
2,0,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,президент египет ввести чрезвычайный положение...,власть египет угрожать ввести страна чрезвычай...,0.898032,0.808042,0.898032,0.667296,0.943043,0.842504,0.066702,0.983709,0.297089,0.848374
3,-1,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,вернуться сирия россиянин волновать вопрос тру...,самолёт мчс вывезти россиянин разрушить сирия,0.675170,0.677628,0.675170,0.542383,0.808040,0.933931,0.659295,0.152258,0.797322,0.662019
4,0,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,москва сирия вернуться 2 самолёт мчс россиянин...,самолёт мчс вывезти россиянин разрушить сирия,0.867209,0.873730,0.867209,0.690162,0.495889,0.512894,0.869615,0.950518,0.844035,0.940833


In [64]:
data.to_csv('data.csv', sep='\t', index=False)

In [70]:
data.head()

,label,text_1,text_2,text_1_norm,text_2_norm,W2V_none,W2V_TF,FT_N_none,FT_N_TF,FT_O_none,FT_O_TF,NMF_cv,NMF_tf,SVD_cv,SVD_tf
0,0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,полицейский разрешить стрелять поражение гражд...,полиция мочь разрешить стрелять хулиган травма...,0.797885,0.772642,0.797885,0.776446,0.892355,0.957886,0.355364,0.980594,0.275518,0.636863
1,0,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,право полицейский проникновение жилища решить ...,правило внесудебный проникновение полицейский ...,0.803657,0.566142,0.803657,0.446428,0.863612,0.702982,0.224124,0.988423,0.084903,0.691494
2,0,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,президент египет ввести чрезвычайный положение...,власть египет угрожать ввести страна чрезвычай...,0.898032,0.808042,0.898032,0.667296,0.943043,0.842504,0.066702,0.983709,0.297089,0.848374
3,-1,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,вернуться сирия россиянин волновать вопрос тру...,самолёт мчс вывезти россиянин разрушить сирия,0.675170,0.677628,0.675170,0.542383,0.808040,0.933931,0.659295,0.152258,0.797322,0.662019
4,0,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,москва сирия вернуться 2 самолёт мчс россиянин...,самолёт мчс вывезти россиянин разрушить сирия,0.867209,0.873730,0.867209,0.690162,0.495889,0.512894,0.869615,0.950518,0.844035,0.940833


In [74]:
X = data.drop(['label','text_1','text_2','text_1_norm','text_2_norm'], axis=1)

In [80]:
y = data['label']

In [76]:
from sklearn.model_selection import cross_val_score

In [92]:
clf = LogisticRegression(C=1000, class_weight='balanced',  multi_class='auto', solver='lbfgs', 
                         max_iter=1000, n_jobs=-1
                        )
cross_val_score(clf, X, y, cv=5).mean()

0.5338037083699848

In [91]:
from sklearn.model_selection import GridSearchCV

In [94]:
lr = LogisticRegression(class_weight='balanced',  multi_class='auto', solver='lbfgs', max_iter=1000)
parameters = {'C':list(np.logspace(-5, 4, 10))}
print (parameters)
clf = GridSearchCV(lr, parameters, cv=5)

{'C': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]}


In [95]:
clf.fit(X, y)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=1000,
          multi_class='auto', n_jobs=None, penalty='l2', random_state=None,
          solver='lbfgs', tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [98]:
pd.DataFrame(clf.cv_results_)

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_C,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,0.038682,0.001470,0.436281,0.440257,1e-05,{'C': 1e-05},10,0.434692,0.440311,0.477540,...,0.486505,0.428744,0.406510,0.446827,0.376039,0.453917,0.006125,0.000207,0.041808,0.009372
1,0.040402,0.001412,0.439740,0.443682,0.0001,{'C': 0.0001},9,0.441603,0.443080,0.479613,...,0.487889,0.430647,0.408587,0.451323,0.380886,0.458240,0.002536,0.000104,0.040848,0.010138
2,0.048564,0.001432,0.463678,0.468035,0.001,{'C': 0.001},8,0.464409,0.469031,0.503110,...,0.513495,0.451401,0.425900,0.479682,0.411357,0.481065,0.001923,0.000103,0.040494,0.011531
3,0.091013,0.001627,0.486094,0.495848,0.01,{'C': 0.01},7,0.490670,0.499135,0.525916,...,0.539100,0.480111,0.437673,0.508214,0.436981,0.505793,0.012688,0.000107,0.042824,0.011023
4,0.128947,0.001456,0.515567,0.525666,0.1,{'C': 0.1},6,0.530753,0.521280,0.539046,...,0.561246,0.513490,0.463989,0.540896,0.482687,0.532941,0.008298,0.000049,0.036350,0.009876
5,0.169685,0.001489,0.532586,0.541890,1,{'C': 1.0},5,0.543884,0.539100,0.563925,...,0.581315,0.527499,0.477147,0.555940,0.496537,0.551790,0.010929,0.000032,0.039632,0.010546
6,0.196818,0.001607,0.533693,0.544139,10,{'C': 10.0},3,0.550104,0.543253,0.566690,...,0.582699,0.530785,0.471607,0.555767,0.497230,0.553865,0.013097,0.000156,0.042284,0.009587
7,0.198439,0.001575,0.533693,0.544658,100,{'C': 100.0},3,0.549413,0.544637,0.568072,...,0.582699,0.530958,0.470222,0.556286,0.497922,0.553346,0.015795,0.000106,0.042742,0.009401
8,0.200084,0.001554,0.533831,0.544623,1000,{'C': 1000.0},1,0.549413,0.544637,0.568763,...,0.582699,0.530958,0.470222,0.556286,0.497922,0.553346,0.017908,0.000064,0.042854,0.009425
9,0.180974,0.001501,0.533831,0.544623,10000,{'C': 10000.0},1,0.549413,0.544291,0.568763,...,0.582699,0.531131,0.470222,0.556286,0.497922,0.553346,0.012585,0.000012,0.042854,0.009352


In [100]:
clf.best_score_

0.5338314653383146

In [101]:
clf.best_params_

{'C': 1000.0}